In [1]:
import numpy as np
import matplotlib.pyplot as plt
from os import listdir
import os
import re
import pandas as pd
import pdb
import scipy.signal

In [2]:
def extractTnew(fname,diagnose=False,diagnose2=False,return_case = 1):
    data = np.loadtxt(fname,delimiter=',',skiprows=1, usecols=(2,3))
    t = data[:,0]
    T = data[:,1]
    
    
    if '111' in fname:
        T = 1.09008 * T + 2.08186
    if '011' in fname:
         T = 1.03963 * T + 5.11961
    
    if '001' in fname:
        T = 1.05498 * T + 2.8904
    
    T = T+273.15
    
    if diagnose:
        plt.plot(t,T)
        plt.show()
    
    #Linear correction
    avg1 = np.mean(T[:10])
    avg2 = np.mean(T[-10:])
    p = np.polyfit((t[0], t[-1]),(avg1, avg2),1)
    T = T - t*p[0] - p[1]
    Tsample = p[1] + p[0]*t[np.round(t.size/2)]
    Tsample = np.mean((avg1,avg2))
    
    
    #Smoothing
    s = 20
    window_length = 9
    poly_order = 3
    #Tsmooth = scipy.signal.savgol_filter(T,window_length,poly_order,mode='nearest')
    #tsmooth = t
    Tsmooth = np.convolve(T,np.ones(s)/s, mode='valid')
    tsmooth = np.convolve(t,np.ones(s)/s, mode='valid')
    
    
    if diagnose:
        plt.plot(t,T,'r')
        plt.plot(tsmooth,Tsmooth, 'b')
        plt.show()

        
    t = tsmooth
    T = Tsmooth
    
    #Differential more useful for finding jumps
    dTdt = np.diff(T) / np.diff(t)
    dTdt = np.convolve(dTdt,np.ones(10),mode='valid')
    
    
    
    if diagnose:
        plt.scatter(t[:-10],dTdt)
        plt.show()
    
    
    #Finding the EC jumps
    if '001' in fname:
        extrema = np.array([np.argmax(dTdt),np.argmin(dTdt)])
        print(fname,' is a 001 file')
    else:
        threshold = 1
        extrema = np.array([np.argmax(dTdt),np.argmin(dTdt)])
        dTdt_copy = np.copy(dTdt)
        for n,_ in np.ndenumerate(dTdt):
            if np.any(np.abs(t[n]-t[extrema])<threshold):
                dTdt_copy[n] = 0
        extrema = np.hstack((extrema,np.array([np.argmax(dTdt_copy),np.argmin(dTdt_copy)])))

    
    if diagnose:
        plt.plot(t,T,'r')
        plt.scatter(t[extrema+1],T[extrema+1],marker='x')
        plt.show()
        
    Tarr = np.array([Tsample])
    for ind in np.sort(extrema):
        tightness = 0.5 # Search region for a maximum and a minimum
        indices = np.where(np.abs(tsmooth-tsmooth[ind])<tightness)
        region = Tsmooth[indices]
        tregion = tsmooth[indices]

        maxmin = np.array([np.max(region), np.min(region)])
        Tarr = np.hstack((Tarr,maxmin))
        maxminarg = np.array([np.argmax(region), np.argmin(region)])
        if diagnose or diagnose2:
            plt.plot(tsmooth,Tsmooth)
            plt.scatter(tregion[maxminarg],maxmin)
            fname = fname.split('\\')[-1]
            plt.title(str(Tarr[0])+' $\degree C$')
            #plt.title(fname)
    if diagnose or diagnose2:
        plt.show()

    if '001' in fname:
        Tarr = np.hstack((Tarr,np.zeros(4)))
    deltaTarr = np.array([Tarr[0],
                     Tarr[1] - Tarr[2],
                     Tarr[3] - Tarr[4],
                     Tarr[5] - Tarr[6],
                     Tarr[7] - Tarr[8]])


    #Check that the results aren't completely ridiculous:
    #if np.max(deltaTarr[1:]) > 0.8: #Change this value when dealing with larger or smaller caloric effects
    #    return False
    #if np.max(deltaTarr[1:]) > 2 * np.min(deltaTarr[1:]):
    #    return False
    
    
    
    if return_case == 1:
        return deltaTarr
    if return_case == 2:
        return Tarr

In [10]:
def extract_info(fname): #Returns an array of three numbers: T,V,box
    if '\\' in fname:
        fname = fname.split('\\')[-1]
    
    pattern = r'([0-9]+) ?K' #Matches any number followed by a K
    p = re.compile(pattern)
    m = p.search(fname)
    T = int(m.groups()[0])
        
    pattern = r'([0-9]+) ?V' #Same but with V
    p = re.compile(pattern)
    m = p.search(fname)
    V = int(m.groups()[0])
    
    pattern = r'box ?([0-9])' #Matches any digit following 'box'
    p = re.compile(pattern,flags=re.IGNORECASE)
    m = p.search(fname)
    box = int(m.groups()[0])
    return np.array([T,V,box])

In [29]:
directory = r'/home/william/Dropbox/Materials Internship/Data/PMN-PT 111 Direct/800V'
def emissivity_correction(directory):
    box_info = np.ones((1,3))*np.pi
    
    for file in listdir(directory):
        if '.csv' not in file:
            continue
        info = extract_info(file)
        
        fname = directory+'/'+file
        data = np.loadtxt(fname,delimiter=',',skiprows=1, usecols=(2,3))
        t = data[:,0]
        T = data[:,1]
        T = T[t<1]
        
        
        
        new_info = np.array([info[2],np.mean(T),info[0]]) #box no., mean temperature, heater temperature
        box_info = np.vstack((box_info,new_info))
    box_info = box_info[1:,:]
    
    #Sort so temperatures are in order
    sorted_indices = np.argsort(box_info[:,2])
    box_info = box_info[sorted_indices,:]
    
    box1 = box_info[box_info[:,0]==1,1]
    box3 = box_info[box_info[:,0]==3,1]
    
    box2 = box_info[box_info[:,0]==2,1]
    
    p1 = np.polyfit(box1,box3,1)
    p2 = np.polyfit(box2,box3,1)
    plt.plot(box1,box3,'x')
    plt.plot(box1,np.polyval(p1,box1))
    plt.ylabel('Sample Temperature (box1) / $\degree$C',size=15)
    plt.xlabel('Black-tape Temperature / $\degree$C',size=15)
    plt.title('Emissivity correction plot')
    plt.show()
    return box_info
    return np.hstack((p1,p2))

coefficients = emissivity_correction(directory)

In [18]:
box_info = coefficients

In [22]:
box_info[box_info[:,0]==1,1]

array([ 140.01153881,  149.96693846,   82.00239214,   88.57741027,
         75.43946   ,  182.34010455,  129.67013859,  159.77534915,
        143.29107027,  105.58304783,  122.87972923,  136.5588641 ,
        109.008475  ,  169.65744531,  156.48966613,   95.33107773,
        126.20978209,  119.38608475,  175.95627755,  146.65424459,
        162.99865909,  153.30723833,  133.06402688,  112.56774937,
        102.10525867,  115.9490623 ])

In [ ]:
#Emissivity correction on 011 - returns the coefficients necessary to do the correction
parent = r'C:\Users\William\Dropbox\Materials Internship\Data\PMN-PT 011 Direct'
directory_list = []
p_values = np.ones((1,4))*np.pi
for directory in listdir(parent):
    if '.' in directory:
        continue
    print('---New directory---: '+directory)
    p = emissivity_correction(parent+'\\'+directory)
    p_values = np.vstack((p_values,p))
    directory_list.append(parent+'\\'+directory)
p_values = p_values[1:]

In [ ]:
#Emissivity correction on 111 - returns the coefficients necessary to do the correction
parent = r'C:\Users\William\Dropbox\Materials Internship\Data\PMN-PT 111 Direct'
#directory_list = []
#p_values = np.ones((1,4))*np.pi
for directory in listdir(parent):
    if '.' in directory:
        continue
    print('---New directory---: '+directory)
    p = emissivity_correction(parent+'\\'+directory)
    p_values = np.vstack((p_values,p))
    directory_list.append(parent+'\\'+directory)

In [7]:
#Test single file (V and T fixed)
directory = r'C:\Users\William\Dropbox\Materials Internship\Data\PMN-PT 001 Direct\200V'
fname = directory+'\\'+r'300316 PMN-PT 001  H2C 390K 200Vbox1.csv'
extractTnew(fname,diagnose=True)

C:\Users\William\Miniconda3\lib\site-packages\ipykernel\__main__.py:26: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
C:\Users\William\Miniconda3\lib\site-packages\matplotlib\collections.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == str('face'):


C:\Users\William\Dropbox\Materials Internship\Data\PMN-PT 001 Direct\200V\300316 PMN-PT 001  H2C 390K 200Vbox1.csv  is a 001 file


array([  3.62659288e+02,   7.51908825e-02,   5.61935188e-02,
         0.00000000e+00,   0.00000000e+00])

In [15]:
#Everything!
parent = r'C:\Users\William\Dropbox\Materials Internship\Data\PMN-PT 111 Direct' #Sample folder
directories = listdir(parent)

#Several voltages:
for n,_ in np.ndenumerate(directories):
    n = n[0] #n is a tuple at first for some reason
    
    #Filtering bad folders/files:
    if '900' in directories[n]:
        continue
    if '800' in directories[n]:# or '200' in directories[n]:
        continue
    if '.' in directories[n]:
        continue
    if 'heating'  in directories[n]:
        continue
    
    directory = parent + '\\' + directories[n] #Voltage directory
    files = listdir(directory)
    
    data = np.zeros(5)
    for file in files:
        #More filtering:
        if '.csv' not in file:
            continue
        info = extract_info(file)
        if info[2] != 1: #Only box 1 allowed
            continue
        
        full = directory+'\\'+file #Complete filename
        try:
            deltaTarray = extractTnew(full)
            if deltaTarray is False:
                continue
            data = np.vstack((data,deltaTarray))
        except:
            print(file)
    data = data[1:,:]
    smoothness = 1
    plt.plot(np.convolve(data[:,0],np.ones(smoothness)/smoothness,mode='valid'),np.convolve(data[:,1],np.ones(smoothness)/smoothness,mode='valid'), 'o-')
    plt.plot(np.convolve(data[:,0],np.ones(smoothness)/smoothness,mode='valid'), np.convolve(-data[:,2],np.ones(smoothness)/smoothness,mode='valid'), 'o-')
#plt.legend(voltages)
#plt.xlim([80, 240])
plt.show()

C:\Users\William\Miniconda3\lib\site-packages\ipykernel\__main__.py:26: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [ ]:
#Test one voltage
directory = r'C:\Users\William\Dropbox\Materials Internship\Data\PMN-PT 111 Direct\300V'
files = listdir(directory)
for file in files:
    if '.csv' not in file:
        continue
    if extract_info(directory+'\\'+file)[2] != 1:
        continue
    try:
        plot_data = extractTnew(directory+'\\'+file,diagnose=True)
        if extractTnew is False:
            print(file)
        plt.scatter(plot_data[0],plot_data[1])
    except:
        pass
        print(file)
plt.show()

In [8]:
#Counting!
samples = [r'C:\Users\William\Dropbox\Materials Internship\Data\PMN-PT 011 Direct',
           r'C:\Users\William\Dropbox\Materials Internship\Data\PMN-PT 111 Direct']
count = 0
for sample in samples:
    for voltage in listdir(sample):
        if '.' in voltage: #Ignore files that are not directories
            continue
        for temp in listdir(sample+'\\'+voltage):
            filename = sample+'\\'+voltage+'\\'+temp
            if '.csv' not in filename:
                continue
            count += 1

print('Done!')
print('Number of files: ',count)

Done!
Number of files:  803


In [9]:
#Writing all of the information to a .csv file
samples = [r'C:\Users\William\Dropbox\Materials Internship\Data\PMN-PT 111 Direct',
           r'C:\Users\William\Dropbox\Materials Internship\Data\PMN-PT 011 Direct']
data = np.ones((count+1,13))*np.pi
n = 0
for sample in samples:
    crystal_orientation = int(sample[-10:-7])
    print('Crystal orientation: ',crystal_orientation)
    for voltage in listdir(sample):
        if '.' in voltage: #Ignore files that are not directories
            continue
        for temp in listdir(sample+'\\'+voltage):
            filename = sample+'\\'+voltage+'\\'+temp
            if '.csv' not in filename:
                continue
            #At this point 'filename' will be a valid T vs t file
            #Now check if the calculation is valid:
            new_data = extractTnew(filename,return_case=2)
            if new_data is False:
                continue
            
            info = extract_info(filename)
            
            #Emissivity correction
            #ind = directory_list.index('\\'.join(filename.split('\\')[:-1]))
            #p = p_values[ind]
            #if info[2] == 1:
            #    new_data = new_data*p[0] + p[1]
            #if info[2] == 2:
            #    new_data = new_data*p[2] + p[3]

            if 'heating' in filename:
                crystal_orientation = crystal_orientation*10+2
            new_row = np.hstack((crystal_orientation,info,new_data))
            if 'heating' in filename:
                crystal_orientation = (crystal_orientation - 2) / 10
                
            data[n] = new_row
            n += 1
        print(voltage)
print('Done!')


Crystal orientation:  111
200V
300V
400V cooling
400V heating
500V
600V
700V
800V
Crystal orientation:  11
030616 PMN-PT 011 200 V
060616 PMN-PT 011 300 V
070616 PMN-PT 011 400 V
070616 PMN-PT 011 500 V
080616 PMN-PT 011 600 V
080616 PMN-PT 011 700V
080616 PMN-PT 011 800 V
080616 PMN-PT 011 900 V
Done!


C:\Users\William\Miniconda3\lib\site-packages\ipykernel\__main__.py:26: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [10]:
df = pd.DataFrame(data)
column_names = ['Crystal orientation', 'T(nominal)/K', 'Voltage/V','Box','T(actual)/K']+['T'+str(n+1) for n in range(8)]
df.columns = column_names
df = df[df.Box != np.pi]

In [11]:
df.head()

,Crystal orientation,T(nominal)/K,Voltage/V,Box,T(actual)/K,T1,T2,T3,T4,T5,T6,T7,T8
0,111.0,385.0,200.0,1.0,354.770844,0.050793,-0.013263,-0.002568,-0.060786,0.058639,-0.004546,0.005182,-0.056748
1,111.0,385.0,200.0,2.0,355.261543,-0.001407,-0.011106,0.045853,-0.013972,-0.002253,-0.062085,0.004107,-0.058549
2,111.0,395.0,200.0,1.0,362.478890,0.071098,0.000588,0.008141,-0.067933,0.067926,-0.004147,0.005423,-0.065260
3,111.0,395.0,200.0,2.0,363.051299,0.067271,0.000008,0.009532,-0.063010,0.067626,-0.005072,0.008011,-0.061105
4,111.0,400.0,200.0,1.0,366.269024,0.077244,-0.002139,-0.002901,-0.081511,0.068961,-0.017007,0.001412,-0.078070


In [12]:
df.to_csv('all_voltages_and_samples.csv',index_label=False,index=False)